In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open('../styles/custom.css', 'r').read()
    return HTML(styles)
css_styling()

# Linear panels and pressure drag

In this final section we will enhance the vortex panel method using linear panels and a pressure drag model so that it can be applied to real engineering flows.

### Linear panels

Up to this point, we've used panels with constant strength because they are simple and effective. Extending the method to use panels with linearly varying strength is only slightly more complex and much more accurate.

![medium](resources\panel_O2.png)

We don't need to consider the detailed derivation (it is similar to the constant strength panels but with more terms). The important point is that the velocity function for a linearly varying strength panel can be written as

$$\vec u_i(x,y) = \gamma_{i-1}(S)\ \vec f_i(x,y,-S)+\gamma_i(S)\ \vec f_i(x,y,S)$$

where $\gamma_i(S)$ is the strength on the positive end of panel $i$, and $\vec f_i(x,y,\pm S)$ are two new influence functions. Just as with the constant-strength panel, the velocity depends __linearly__ on $\gamma$, and for $N$ panels we have only $N$ unknowns. Therefore, we can construct an $N$ by $N$ linear system.

---

##### Quiz

Approximately how many constant strength panels will we need to match the accuracy of $N$ linearly varying panels?

1. $N$
1. $2N$
1. $N^2$


---

Lets test it out using our old friend the 3-panel triangle and using `VortexPanel.solve_gamma_O2` to solve for the linearly varying strength on each panel. This function has exactly the same input arguments as `solve_gamma`, and the resulting panel array still works with `plot_flow`, `get_array`, etc.


In [ ]:
from matplotlib import pyplot
%matplotlib inline
import numpy
import VortexPanel as vp
import BoundaryLayer as bl

In [ ]:
%time
triangle = vp.make_polygon(N=3,sides=3)
vp.solve_gamma(triangle)
vp.plot_flow(triangle,vmax=3)
pyplot.show()

triangle2 = vp.make_polygon(N=3,sides=3)
vp.solve_gamma_O2(triangle2)  ## O2 !!
vp.plot_flow(triangle2,vmax=3)

As you can see, the panels with linearly varying strength give a __much__ better result for a given $N$. This is because the error between the true analytic $\gamma(s)$ and a piecewise constant approximation is $\epsilon\sim 1/N$, whereas it is $\epsilon\sim (1/N)^2$ for a piecewise linear approximation. Because of the exponent, this is called _second-order_ or _O(2)_ error. 

This is worth saying again: We are __massively__ reducing the error without solving __any__ extra equations!

##### Numerical fundamental: O(2)
##### Second-order methods are unreasonably effective

This improvement is fairly general and second-order methods should be used whenever possible. For example, `BoundaryLayer.march` uses a second-order ODE integration method; similar to the trapezoidal rule. Increasing beyond O(2) is possible, but often incurs large additional complexity and computational effort.

Also note that I've used `%time` in the code above to time how long this takes to run. Use it to compare the speed of the two methods.

---

### Bernoulli equation

Now that we've turbo-charged the solver, lets use this to find the drag!

The first step is to compute the pressure, $p$, which we can get using the Bernoulli equation

$$ p+\frac 12 \rho\ |u|^2-\rho gz = B $$

where $\rho$ is the density, $\rho gz$ is the head, and $B$ is the Bernoulli coefficient. As in the rest of this course, will assume no hydrostatic pressure.

Note that the Bernoulli coefficient is kind of a pain. For example, the pressure far from the body can't be put in terms of the velocity alone, ie:

$$p_\infty = B-\frac 12 \rho U^2$$


To avoid the ambiguity of the Bernoulli coefficient, we typically work with the pressure coefficient $c_p$

$$c_p = \frac{p-p_\infty}{\frac 12\rho\ U^2}$$
instead of the pressure.

##### Quiz

How can we simplify the pressure coefficient when there is no hydrostatic pressure?

1. $c_p = \frac 12 |u|^2$
1. $c_p = 1-|u|^2/U^2$
1. $c_p = 1-p/p_\infty$

---
So lets plot $c_p(s)$ for the flow over a circle using this equation:

In [ ]:
circle = vp.make_circle(N=32)  # set-up geom
s = vp.distance(circle)

vp.solve_gamma(circle)         # find gamma O(1)
pyplot.plot(s,1-vp.get_array(circle,'gamma')**2, label='O(1)')

vp.solve_gamma_O2(circle)      # find gamma O(2)
pyplot.plot(s,1-vp.get_array(circle,'gamma')**2, label='O(2)')

pyplot.plot(s,2*numpy.cos(2.*s)-1, label='analytic')
pyplot.xlabel(r'$s/R$', fontsize=18)
pyplot.ylabel(r'$c_p$', fontsize=18)
pyplot.legend()

Where we've use $|u|^2=\gamma^2$ on the body (yet again), and threw in another comparison between the solvers for good measure. 


Notice the stagnation pressure coefficient is one by definition, and that $c_p$ is symmetric on the front and back of the body.

---

## Pressure force

Once we have the surface pressure we can use it to determine the pressure force on the body, as

$$ \vec F_p = -\oint p \hat n da $$

where $\oint da$ is the integral over the body surface, and $\hat n$ is the normal vector to the surface. 

For instance, the lift coefficient is then:

$$ C_L = \frac{\vec F_p \cdot \hat U_\perp} { \tfrac 12 \rho U^2 A }  = \frac{-\oint p \hat n ds \cdot \hat U_\perp}{ \tfrac 12 \rho U^2 c } c= -\frac{\oint c_p (s_x\cos\alpha+s_y\sin\alpha) ds}{ c } $$

where $c$ is the coord, $\hat U_\perp=\cos\alpha\hat y-\sin\alpha\hat x$ is the direction perpendicular to the free stream and $\hat n=s_x\hat y-s_y\hat x$ as seen in the figure below:

![short](resources/graphics6.png)

Let's test is on the Jukowski foil...

In [ ]:
def C_L(panels,alpha):
    gamma, xc, S, sx, sy = vp.get_array(panels,'gamma','xc','S','sx','sy')
    c = max(xc)-min(xc)
    perp = sx*numpy.cos(alpha)+sy*numpy.sin(alpha)
    return -sum((1-gamma**2)*2*S*perp)/c

foil = vp.make_jukowski(N=32)
vp.solve_gamma_kutta(foil,alpha=0.1)
print("C_L = {:.3f}".format(C_L(foil,alpha=0.1)))

This function will give the same result for the __total__ lift as the circulation-based formula from notebook 3_3. However, when simulating the flow on multiple bodies, only the formula above can be applied to find the lift on each body individually.


---

Similarly, the pressure drag coefficient $C_P$ is:

$$ C_P = \frac{\vec F_p \cdot \hat U_\parallel} { \tfrac 12 \rho U^2 A } = \frac{\oint c_p (s_y\cos\alpha-s_x\sin\alpha) ds}{ t } $$

where $t$ is the thickness and $\hat U_\parallel$ is the direction parallel to $\vec U$. Let's test $C_P$ on the circle flow with $\alpha=0$.

In [ ]:
def C_P(panels):
    gamma,yc,sy,S = vp.get_array(panels, 'gamma', 'yc', 'sy', 'S')
    t = max(yc)-min(yc)
    return sum((1-gamma**2)*sy*2*S)/t

print("C_P = {:.18f}".format(C_P(circle)))

##### Quiz

Why is the pressure drag equal to zero?

1. The symmetric shape of the circle
1. Potential flow
1. Insufficient resolution
---

## Separated flow

Here is a sketch of the pressure on a circular cylinder in three different flow conditions:

![short](resources/graphics5.png)

---

We ran into D'Alembert's paradox earlier, which states that there is zero drag force in potential flow, regardless of the body shape. The symmetric pressure makes this unavoidable. 

The boundary layer in a real (viscous) fluid induces friction force. These forces tend to be small, but they lead to flow separation, after which the potential flow solution is completely invalid. Also note that the delayed separation in a turbulent boundary layer completely changes the wake pressure as well.


##### Fluid fundamental:
##### The pressure in the wake - and therefore the drag - is highly sensitive to separation


Unfortunately, we can't use boundary layer theory to solve for the pressure (or anything else) after separation. We'll need another way to get the drag:

---



## Momentum deficit

Instead of dealing with pressure, I prefer think of D'Alembert's paradox in terms of flow momentum. The momentum in the flow behind the circle is exactly the same as the momentum in front of the circle; therefore the drag is zero. 

##### Fluids fundamental:
##### The wake momentum deficit is equal to the drag.

Is there a way we can approximate the wake deficit using the tools we have?

Lets assume the boundary layers separate from the body into a pair of **shear layers**, but maintain the same strength ($\gamma$) as they were at the point of separation. We'll also assume that the distance between the shear layers ($w$) stays the same.

![short](resources/shear_wake.png)

Note: the real wake doesn't look like this. But these assumptions might be good enough to determine how the drag __scales__.


---

##### Quiz

In terms of length and velocity scales, what is the momentum deficit induced by the shear layers in the sketch?

1. $\rho U^2 t$
1. $\rho \gamma^2 w$
1. $\rho U^2 \gamma $

---

Let's do a quick sanity check on this model to make sure we're happy with it:
- The wider the wake, the greater the drag
- The stronger the shear layers, the greater the drag
- __Critically__ we can compute these values ($\gamma$,$w$) using numerical boundary layer decomposition

This is promising, but we'll need data to make a quantitative assessment of the model.

---

## Wake coefficient calibration

Here is some data from Hoerner on the total drag coefficient of elliptical cylinders:

In [ ]:
c_t = [1.27,1.67,2.06,3.06,3.39,4.03,4.73,8.15]
CD = [0.919,0.718,0.604,0.438,0.408,0.342,0.324,0.267]
pyplot.scatter(c_t,CD)
pyplot.xlabel(r'$c/t$', fontsize=18)
pyplot.ylabel(r'$C_D$', fontsize=18)

To test (and potentialy calibrate) our simple wake deficit model against this data we first define a wake momentum coefficient 
$$C_\gamma = \frac w t \frac {\gamma^2}{U^2} $$
Note that I've scaled the wake deficit estimate by $\rho U^2 t$, similar to the pressure drag scaling.

Then `for` each $c/t$:
1. `Solve` the flow, `split` the body, and `march` the boundary layer.
1. Measure the predicted friction coefficient $C_F$.
1. Measure the predicted $\gamma$ and $w$ at the separation point to determine $C_\gamma$.
1. Use $C_F$ and the experimental drag to compute the pressure drag coefficient $C_P=C_D-C_F \frac ct $.
1. Finally, plot $C_\gamma$ against $C_P$, giving:

![medium](resources/pressure_drag.png)

Despite simplicity of our wake model, the wake coefficient is almost perfectly correlated with the pressure drag! The equation for the line above is:

$$ C_P = 0.343 C_\gamma$$

## Summary

We can estimate pressure drag forces on general bodies by 
- using the `VortexPanel` and `BoundaryLayer` to determine $C_\gamma$ for a given geometry, and
- using the empirical relation above to estimate $C_P$.

---

##### Quiz

What do you think would be the fastest way to estimate forces in the early stages of the design process?

1. Analytic potential flow theory
1. Numerical boundary layer decomposition
1. Navier-Stokes simulations (CFD)
2. Experiments

---


##### Your turn #6

 - **Complete** the function `C_gamma` to determine the wake coefficient for a given array of panels
 - **Time** how long it takes for you to predict $C_P$ for $t/c$=`linspace(0.1,1.1,10)` using `%time` 
 - **Plot and Compare ** your predictions to Hoerner's reported $C_D$ data.

---

##### Solution #6


In [ ]:
def C_gamma(panels):
    return # your code here

In [ ]:
%time
t_c = numpy.linspace(0.1,1.1,20)
# your code here

In [ ]:
c_t = [1.27,1.67,2.06,3.06,3.39,4.03,4.73,8.15]
CD = [0.919,0.718,0.604,0.438,0.408,0.342,0.324,0.267]
pyplot.scatter(c_t,CD,label='experimental C_D')

# your code here

pyplot.legend()
pyplot.xlabel(r'$c/t$', fontsize=18)
pyplot.ylabel(r'Drag', fontsize=18)